In [1]:
import pandas as pd
df = pd.read_csv(r"http://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv")

In [2]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


### basic preprocessing

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Text preprocessing functions
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    return " ".join([word.lower() for word in text.split()])

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    return re.sub('\s+', ' ', text).strip()

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_small_sentences(df):
    df['content'] = df['content'].apply(lambda x: np.nan if len(str(x).split()) < 3 else x)
    return df

def normalize_text(df):
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        df = remove_small_sentences(df)
        return df.dropna(subset=['content'])
    except Exception as e:
        print(e)
        return df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iampr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\iampr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = normalize_text(df)

In [6]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [8]:
df.sample(5)

,tweet_id,sentiment,content
1767,1957392626,0,itstayloryall im sad missed family reunion
37228,1753450968,0,asked sing yet again aha coffee mornin time ps...
5360,1960762036,0,lolzyluvsjb neither could i looked ebay like e...
32016,1752157659,1,eplisa diaper feeding cooking stuff hahaha yea...
30594,1751644693,1,justinsxe woot woot super cool


### apply BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 10)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

#### logging experiment on mlflow


In [10]:
import dagshub
import mlflow

dagshub.init(repo_owner='iamprashantjain', repo_name='Emotion-Detection-MLOps', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/iamprashantjain/Emotion-Detection-MLOps.mlflow")
mlflow.set_experiment("baseline model")

Accessing as iamprashantjain

Initialized MLflow to track repo "iamprashantjain/Emotion-Detection-MLOps"

Repository iamprashantjain/Emotion-Detection-MLOps initialized!

2025/07/20 22:44:54 INFO mlflow.tracking.fluent: Experiment with name 'baseline model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c746a5e75424444387ff33ca4afbd667', creation_time=1753031695025, experiment_id='1', last_update_time=1753031695025, lifecycle_stage='active', name='baseline model', tags={}>

#### run mlflow experiment

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with mlflow.start_run():
    # Log preprocessing params
    mlflow.log_param("vectorizer", "BOW")
    mlflow.log_param("num_features", 10)
    mlflow.log_param("test_size", 0.2)

    # Model building & training
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Log model params
    mlflow.log_param("model", "logistic regression")

    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Optionally log the model
    mlflow.sklearn.log_model(model, "model")

    # Log the notebook
    import os
    notebook_path = "experiments.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace{notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Print results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

i:\CampusX_DS\campusx_dsmp2\9. MLOps revisited\tutorial\Emotion-Detection-MLOps\venv\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


Accuracy: 0.5667
Precision: 0.6487
Recall: 0.2812
F1 Score: 0.3924
